In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError
from qiskit import available_backends, register, execute, get_backend
from qiskit.tools.visualization import circuit_drawer, plot_histogram, plot_state
from qiskit.tools.qi.qi import state_fidelity
import random, pickle, time, datetime, xlsxwriter
import numpy as np
import scipy.io as spio
import importlib
import circuit_builder
importlib.reload(circuit_builder)
import encoding_schemes
importlib.reload(encoding_schemes)

#setup
import sys, getpass
try:
    sys.path.append("../../") # go to parent dir
    import Qconfig
    qx_config = {
        "APItoken": Qconfig.APItoken,
        "url": Qconfig.config['url']}
    print('Qconfig loaded from %s.' % Qconfig.__file__)
    print()
except:
    APItoken = getpass.getpass('Please input your token and hit enter: ')
    qx_config = {
        "APItoken": APItoken,
        "url":"https://quantumexperience.ng.bluemix.net/api"}
    print('Qconfig.py not found in qiskit-tutorial directory; Qconfig loaded using user input.')
    
#matlab stuff
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from numpy import matrix
from math import pi

#set 
QX_URL = "https://quantumexperience.ng.bluemix.net/api"
QX_TOKEN = "06704f99fc6dd68ab5ae2663f0861723406de8b4553cacdabdf500292c937717442daa90c2ffa19109cfb9114adb7c87711a08860d34d1da9332ac664e4e24a7"

#register token and url
register(QX_TOKEN, QX_URL)

# query for available backends
print("Available backends: ", available_backends())

backend = "ibmqx4"

# This function deals with the case when circuits contains a length one circuit, which is parsed
# as a string rather than an array. If we find a string we just return it
def hasString(circuits):
    strings = []
    newArray = []
    for element in circuits:
        if isinstance(element,np.ndarray):
            newArray.append(element)
            continue
        else:
            strings.append(element)
    strings = np.asanyarray(strings)
    return [strings[0]]
    
        

# This function is not robust. Ties between circuits of equal length are broken arbitrarily.
# Sometimes, it may matter, because two implementations that are the same length may involve
# different numbers of gates and may have different performances. However, from current data,
# it seems that the shortest circuit is usually unique, so this is not currently a concern.
def getShortest(circuits):
    circuit = hasString(circuits)
    if circuit is None:
        circuit = min(circuits, key = len)
    return [circuit]

# code for extracting gate constructions
extract = spio.loadmat('shortnormgates.mat', squeeze_me = True)
CZ = extract['CZnorm']

# IMPORTANT SETTINGS
#
# Update the scheme to reflect your encoding scheme
scheme = encoding_schemes.FourTwoTwo()

# Update desired outputs to reflect your circuit
desiredOutputs = [["0000","1111"], ["0101","1010"], ["1100","0011"],["0110","1001"]]
#desiredOutputs = [["00"], ["10"], ["01"], ["11"]]

# Setup variables
initialStates = [0,1,2,3]
reps = 9

# Specify circuits
circuits = CZ
circuits = getShortest(circuits)
print(circuits)
# Try this one (replace CZ as appropriate) if you only want to find the desiredOutputs for a given circuit
# circuits = [CZ[0]]
#


# These update automatically with scheme
n = scheme.n
validOutputs = scheme.validOutputs

startTime = time.time()

# Main benchmarking procedure
for circuit in circuits:
    
    for compiler in initialStates:
        print("Circuit is: "+str(circuit))
        print("Compiler is: "+str(compiler))
            
        avgSuccess = 0

        for trial in range(reps):
            
            # Standard setup
            qr = QuantumRegister(n)
            cr = ClassicalRegister(n)
            qc = QuantumCircuit(qr,cr)
            
            scheme.prepareZeros(qc)
            
            # Maps the input info into actual physical qubits
            perm = scheme.perm
            scheme.setPerm()
            
            # Sets our initial state
            scheme.setInput(qc, compiler)
            
            
            # Build the circuit
            registerPerm = circuit_builder.buildCirc(qc, circuit, perm)
            
            for i in range(n):
                qc.measure(qr[registerPerm[i]],cr[i])
            
            # Standard execution code
            job = execute(qc, backend)
            result = job.result()
            data = result.get_counts(qc)
            
            # Prints raw data
            print(data)
            
            # Clean and process data
            successes = 0
            totalValid = 0
            for key, value in data.items():
                if key in desiredOutputs[compiler]:
                    successes += value
                if key in validOutputs:
                    totalValid += value
            
            # If we hit this, something went very wrong
            if totalValid == 0:
                print("ERROR WITH VALID INPUTS")
                
            else:
                # Comment out if running more trials
                # print("Success rate is: "+ str(successes/totalValid))
                avgSuccess += successes/totalValid
        
        # Calculate and display information
        avgSuccess /= reps
        print("Average of success rates is: "+ str(avgSuccess)+ "\n")

endTime = time.time()
print("Time running: "+str(endTime-startTime))

Qconfig loaded from C:\Users\ddrsq\OneDrive\IBM Q\qecc_benchmark\Qconfig.py.

Available backends:  ['ibmqx2', 'ibmq_qasm_simulator', 'local_qasm_simulator', 'local_clifford_simulator', 'ibmqx4', 'local_statevector_simulator', 'ibmqx5', 'local_unitary_simulator']
[['P 1  2  3  4']]
Circuit is: ['P 1  2  3  4']
Compiler is: 0
{'0000': 428, '0001': 24, '0010': 17, '0011': 18, '0100': 11, '0101': 5, '0110': 6, '0111': 53, '1000': 6, '1001': 4, '1010': 2, '1011': 23, '1100': 32, '1101': 32, '1110': 72, '1111': 291}


Got a 400 code response to https://quantumexperience.ng.bluemix.net/api/Jobs/5b5215c8cb18f50037619a89?access_token=6QBfkq6Z4gBwW6VBXgZcYuAhsnEza3UKuZGnUCA3E1O0SzAAejfDU89p7IVk1u0F: {"error":{"status":400,"message":"Generic error","code":"GENERIC_ERROR"}}
Got a 400 code response to https://quantumexperience.ng.bluemix.net/api/Jobs/5b5215c8cb18f50037619a89?access_token=6QBfkq6Z4gBwW6VBXgZcYuAhsnEza3UKuZGnUCA3E1O0SzAAejfDU89p7IVk1u0F: {"error":{"status":400,"message":"Generic error","code":"GENERIC_ERROR"}}


{'0000': 439, '0001': 15, '0010': 13, '0011': 21, '0100': 9, '0101': 3, '0110': 13, '0111': 37, '1000': 18, '1001': 3, '1010': 1, '1011': 11, '1100': 43, '1101': 31, '1110': 80, '1111': 287}
{'0000': 429, '0001': 13, '0010': 14, '0011': 20, '0100': 9, '0101': 4, '0110': 10, '0111': 37, '1000': 13, '1001': 1, '1010': 9, '1011': 11, '1100': 40, '1101': 35, '1110': 85, '1111': 294}
{'0000': 422, '0001': 11, '0010': 16, '0011': 26, '0100': 5, '0101': 6, '0110': 12, '0111': 31, '1000': 15, '1001': 1, '1010': 4, '1011': 20, '1100': 32, '1101': 30, '1110': 100, '1111': 293}
{'0000': 432, '0001': 8, '0010': 11, '0011': 25, '0100': 1, '0101': 4, '0110': 10, '0111': 35, '1000': 12, '1001': 6, '1010': 2, '1011': 19, '1100': 34, '1101': 46, '1110': 102, '1111': 277}
{'0000': 422, '0001': 12, '0010': 12, '0011': 22, '0100': 5, '0101': 5, '0110': 14, '0111': 26, '1000': 11, '1001': 5, '1010': 4, '1011': 14, '1100': 42, '1101': 34, '1110': 101, '1111': 295}


Got a 502 code response to https://quantumexperience.ng.bluemix.net/api/Jobs/5b5217af11322d003bf0fd81?access_token=6QBfkq6Z4gBwW6VBXgZcYuAhsnEza3UKuZGnUCA3E1O0SzAAejfDU89p7IVk1u0F: 502 Bad Gateway: Registered endpoint failed to handle the request.



{'0000': 395, '0001': 8, '0010': 9, '0011': 21, '0100': 8, '0101': 5, '0110': 14, '0111': 25, '1000': 26, '1001': 2, '1010': 8, '1011': 25, '1100': 35, '1101': 37, '1110': 107, '1111': 299}
{'0000': 415, '0001': 9, '0010': 22, '0011': 25, '0100': 8, '0101': 6, '0110': 15, '0111': 49, '1000': 9, '1001': 5, '1010': 3, '1011': 14, '1100': 43, '1101': 49, '1110': 88, '1111': 264}
